In [3]:
import pandas as pd
import re
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# =========================
# CONFIGURATION
# =========================
API_KEY = "YOUR_YOUTUBE_API_KEY"   # <-- put your key here
CSV_PATH = "vdoLinks.csv"          # must contain columns: youtubeId, title

youtube = build("youtube", "v3", developerKey=API_KEY)

# =========================
# HELPER FUNCTIONS
# =========================
def clean_text(text):
    """Remove noisy characters but keep readable comments"""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def fetch_comments(video_id, max_results=5):
    """Fetch top-level YouTube comments safely"""
    try:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=max_results,
            textFormat="plainText"
        )
        response = request.execute()

        comments = []
        for item in response.get("items", []):
            comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            comments.append(clean_text(comment))

        return comments

    except HttpError as e:
        if "commentsDisabled" in str(e):
            return ["[Comments are disabled for this video]"]
        return [f"[Error fetching comments: {e.reason}]"]

# =========================
# MAIN LOGIC
# =========================
movies = pd.read_csv(CSV_PATH, dtype=str)

for _, row in movies.iterrows():
    title = row["title"]
    video_id = row["youtubeId"]

    print("\n" + "=" * 80)
    print(f"Movie Name: {title}")

    comments = fetch_comments(video_id)

    if not comments:
        print("No comments found.")
    else:
        print("\nComments:")
        for i, c in enumerate(comments, 1):
            print(f"{i}. {c}")


ModuleNotFoundError: No module named 'googleapiclient'